# Exercício: Regressão
Objetivos: Exercitar os conceitos referente à regressão.

1. Crie um arquivo Jupyter Notebook e realize as seguintes operações:

### a. Ler o dataset fakeTelegram.BR_2022.csv

### b. Remova os trava-zaps, as linhas repetidas (duplicadas) e textos com menos de 5 palavras.

In [1]:
import duckdb

conn = duckdb.connect()

telegram_data = conn.read_csv("../data/fakeTelegram.BR_2022.csv")

query = """
    SELECT * FROM telegram_data
"""

df = conn.execute(query).fetchdf()

In [2]:
df.head()

,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,message_type,messenger,media_name,media_md5
0,2022-10-05 06:25:04,1078cc958f0febe28f4d03207660715f,12283e08a2eb5789201e105b34489ee7,None,None,None,False,False,False,Então é Fato Renato o áudio que eu ouvi no wha...,5,2022-10-05 06:25:28.863641,0.0000,NaN,16385,Texto,telegram,None,None
1,2022-10-05 06:25:08,None,12283e08a2eb5789201e105b34489ee7,None,None,None,False,False,False,"Saiu no YouTube do presidente a 8 horas atrás,...",5,2022-10-05 06:25:28.926311,0.0644,NaN,16386,Texto,telegram,None,None
2,2022-10-05 06:26:28,92a2d8fd7144074f659d1d29dc3751da,9f2d7394334eb224c061c9740b5748fc,None,None,None,False,False,False,"É isso, nossa parte já foi quase toda feita. N...",5,2022-10-05 06:26:29.361949,-0.3551,0.157242,16366,Texto,telegram,None,None
3,2022-10-05 06:27:28,d60aa38f62b4977426b70944af4aff72,c8f2de56550ed0bf85249608b7ead93d,94dca4cda503100ebfda7ce2bcc060eb.jpg,image/jpg,None,True,False,False,GENTE ACHEI ELES EM UMA SEITA MAÇONÁRICA,5,2022-10-05 06:27:29.935624,0.0000,NaN,19281,Imagem,telegram,None,94dca4cda503100ebfda7ce2bcc060eb
4,2022-10-05 06:27:44,cd6979b0b5265f08468fa1689b6300ce,e56ec342fc599ebb4ed89655eb6f03aa,5ad5c8bbe9da93a37fecf3e5aa5b0637.jpg,image/jpg,None,True,False,False,None,5,2022-10-05 06:28:29.316325,NaN,NaN,507185,Imagem,telegram,None,5ad5c8bbe9da93a37fecf3e5aa5b0637


In [3]:
df.shape

(557586, 19)

In [4]:
df = conn.execute(f"""
    SELECT * 
    FROM telegram_data
    WHERE trava_zap IS NOT TRUE
""").fetch_df()

In [5]:
df.shape

(557570, 19)

In [6]:
df_ = conn.execute("SELECT DISTINCT * FROM df").fetch_df()

In [7]:
query = """
SELECT *
FROM df_
WHERE array_length(string_split(text_content_anonymous, ' ')) >= 5
"""

df = conn.execute(query).fetch_df()

df.shape

(336944, 19)

### c. Agrupe as linhas com postagens iguais ou extremamente semelhantes. Aqui você pode utilizar uma métrica de semelhança de textos. Crie uma variável para representar a quantidade de vezes que a mensagem foi compartilhada. Observe que ao agrupar linhas que possuem a “mesma” postagem (texto), você deve escolher como valor para as variáveis data e hora da postagem, os valores da cópia mais antiga.

In [ ]:
import unicodedata
import re
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity

def clean_text(text):
    if not isinstance(text, str):
        return ""
    
    text = text.lower()
    
    # remover acentos
    text = unicodedata.normalize('NFKD', text)\
           .encode('ascii', 'ignore')\
           .decode('utf-8')
    
    # remover pontuação e caracteres especiais
    text = re.sub(r'[^\w\s]', '', text)
    
    # remover emojis
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # símbolos & pictogramas
        u"\U0001F680-\U0001F6FF"  # transporte & símbolos
        u"\U0001F1E0-\U0001F1FF"  # bandeiras (iOS)
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    text = ' '.join(text.split())
    
    return text

In [9]:
df['cleaned_text'] = df['text_content_anonymous'].apply(clean_text)


In [10]:
!pip install -U datasketch

In [11]:
from datasketch import MinHash, MinHashLSH
import pandas as pd

texts = df['cleaned_text'].tolist()

minhashes = {}
for idx, text in enumerate(texts):
    mh = MinHash(num_perm=128)
    for word in text.split():
        mh.update(word.encode('utf-8'))
    minhashes[idx] = mh


lsh = MinHashLSH(threshold=0.5, num_perm=128)

for idx, mh in minhashes.items():
    lsh.insert(idx, mh)

groups = []
used = set()

for idx in minhashes:
    if idx not in used:
        similar_indices = lsh.query(minhashes[idx])
        
        groups.append(similar_indices)
        used.update(similar_indices)

grouped_data = []
for group in groups:
    group_df = df.iloc[list(group)]
    oldest = group_df.loc[group_df['date_message'].idxmin()]
    
    count = len(group_df)
    
    new_row = oldest.to_dict()
    new_row['shared_count'] = count
    grouped_data.append(new_row)

result_df = pd.DataFrame(grouped_data)

In [12]:
result_df = result_df.sort_values('date_message')
result_df.head()

,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,...,date_system,score_sentiment,score_misinformation,id_message,message_type,messenger,media_name,media_md5,cleaned_text,shared_count
9354,2022-09-29 00:00:04,f0ac73b52a8c343778e3d68682d6f529,857cd5311da1bdc15eb9e6918a47c6c6,None,None,None,False,False,False,Bitcoin has really changed my life for the bes...,...,2022-10-03 04:04:31.751964,0.0000,0.101220,873367,Texto,telegram,None,None,bitcoin has really changed my life for the bes...,2
12252,2022-09-29 00:00:07,3b94e832cae91837c425363cd63fdfb0,959f13e0079883060632c74ffc81c547,None,None,None,False,False,False,"é, mas o circo tá armado pra fazer aquele espe...",...,2022-10-03 04:04:31.881104,-0.8885,0.271192,9290,Texto,telegram,None,None,e mas o circo ta armado pra fazer aquele espet...,1
82790,2022-09-29 00:00:11,None,2ff252ad4422e11a6a8abfaa747abb55,None,None,t.me/alexeconomia,False,True,False,BOE TOMOU MEDIDAS DE EMERGÊNCIA NA QUARTA-FEIR...,...,2022-10-03 04:04:32.024363,-0.7269,0.065431,47690,Texto,telegram,None,None,boe tomou medidas de emergencia na quartafeira...,1
12253,2022-09-29 00:00:13,61606aac57503ba74c73c757039b460b,b11f2df64ac19aad47a50accf32052d6,None,None,None,False,False,False,Caramba! Por isso o meu não compartilha nem to...,...,2022-10-03 04:04:32.128902,0.0000,NaN,151069,Texto,telegram,None,None,caramba por isso o meu nao compartilha nem tod...,2
9831,2022-09-29 00:00:15,abe534d581ec6d552243d6955d3c3cd8,92bb16424091252ba82616cde38e1c19,None,None,None,False,False,False,"Welcome, Suely Pimentel\n\n🔸 [USER] — professi...",...,2022-10-03 04:04:32.233217,0.0000,NaN,39817,Texto,telegram,None,None,welcome suely pimentel user professional tool ...,4618


### d. Você pode criar novos atributos numéricos, tais como: quantidade de palavras, quantidade de caracteres etc.

In [13]:
import re

def extract_features(text):
    if not isinstance(text, str):
        return {
            'word_count': 0,
            'char_count': 0,
            'unique_word_ratio': 0,
            'link_count': 0,
            'avg_word_length': 0
        }
    
    words = text.split()
    word_count = len(words)
    char_count = len(text)
    
    unique_words = set(words)
    unique_word_ratio = len(unique_words) / word_count if word_count > 0 else 0
    
    link_count = len(re.findall(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text))
    
    avg_word_length = sum(len(word) for word in words) / word_count if word_count > 0 else 0
    
    return {
        'word_count': word_count,
        'char_count': char_count,
        'unique_word_ratio': unique_word_ratio,
        'link_count': link_count,
        'avg_word_length': avg_word_length
    }

features = result_df['cleaned_text'].apply(lambda x: pd.Series(extract_features(x)))

result_df = pd.concat([result_df, features], axis=1)

print(result_df[['cleaned_text', 'shared_count', 'word_count', 'char_count', 
                'unique_word_ratio', 'link_count', 'avg_word_length']].head())

                                            cleaned_text  shared_count  \
9354   bitcoin has really changed my life for the bes...             2   
12252  e mas o circo ta armado pra fazer aquele espet...             1   
82790  boe tomou medidas de emergencia na quartafeira...             1   
12253  caramba por isso o meu nao compartilha nem tod...             2   
9831   welcome suely pimentel user professional tool ...          4618   

       word_count  char_count  unique_word_ratio  link_count  avg_word_length  
9354         72.0       366.0           0.666667         0.0         4.097222  
12252        28.0       142.0           0.857143         0.0         4.107143  
82790        34.0       207.0           0.882353         0.0         5.117647  
12253        19.0       102.0           0.947368         0.0         4.421053  
9831         10.0        74.0           1.000000         0.0         6.500000  


# Utilizando os dados referente a postagens no Telegram, crie um modelo preditivo (regressor) para, dado os dados de uma postagem, prever a quantidade de compartilhamentos dessa mensagem, o que é denominado potencial de “viralização”.

## A avaliação experimental deverá considerar:

e. O modelo de regressão: regressão linear multivariada, regressão polinomial
multivariada;

f. Explorar funções: exponencial, seno, cosseno (OPCIONAL);

g. Regularização: Com regularização (Ridge, Lasso ou ElasticNet) e sem
regularização;

h. Normalização dos dados: sem normalização, Z-Score, Min-Max (OPCIONAL);

i. Pré-processamento de dados: sem pré-processamento e com pré-processamento;

j. Embedding: BOW, TF-IDF, Word2Vec;

k. N-Gramas: unigramas, bigramas, trigramas;

l. Treinamento, Validação e Teste: Outer K-Fold Cross-Validation;

In [14]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, FunctionTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline

numeric_features = ['word_count', 'char_count', 'unique_word_ratio', 
                   'link_count', 'avg_word_length']

X = result_df[numeric_features]
y = np.log1p(result_df['shared_count'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### item e.

In [15]:
lr_model = Pipeline([
    ('regressor', LinearRegression())
])
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)
print("Linear Regression MSE:", mean_squared_error(y_test, y_pred_lr))

poly_model = Pipeline([
    ('poly_features', PolynomialFeatures(degree=2, include_bias=False)),
    ('regressor', LinearRegression())
])
poly_model.fit(X_train, y_train)
y_pred_poly = poly_model.predict(X_test)
print("Polynomial Regression MSE:", mean_squared_error(y_test, y_pred_poly))


Linear Regression MSE: 0.3789330706422574
Polynomial Regression MSE: 0.37830116960476035


### item f.

### item g.

In [17]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet

ridge = Ridge(alpha=1.0).fit(X_train, y_train)
lasso = Lasso(alpha=0.1).fit(X_train, y_train)
elastic = ElasticNet(alpha=0.1, l1_ratio=0.5).fit(X_train, y_train)

print("Ridge MSE:", mean_squared_error(y_test, ridge.predict(X_test)))
print("Lasso MSE:", mean_squared_error(y_test, lasso.predict(X_test)))
print("ElasticNet MSE:", mean_squared_error(y_test, elastic.predict(X_test)))


Ridge MSE: 0.3789332050551602
Lasso MSE: 0.3799010009679673
ElasticNet MSE: 0.37977058145294446


### Item h.

In [18]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

results = {}

pipeline_none = Pipeline([
    ('regressor', LinearRegression())
])
pipeline_none.fit(X_train, y_train)
y_pred_none = pipeline_none.predict(X_test)
results['sem_normalizacao'] = {
    'MSE': mean_squared_error(y_test, y_pred_none),
    'R2': r2_score(y_test, y_pred_none)
}

pipeline_zscore = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])
pipeline_zscore.fit(X_train, y_train)
y_pred_zscore = pipeline_zscore.predict(X_test)
results['zscore'] = {
    'MSE': mean_squared_error(y_test, y_pred_zscore),
    'R2': r2_score(y_test, y_pred_zscore)
}

pipeline_minmax = Pipeline([
    ('scaler', MinMaxScaler()),
    ('regressor', LinearRegression())
])
pipeline_minmax.fit(X_train, y_train)
y_pred_minmax = pipeline_minmax.predict(X_test)
results['minmax'] = {
    'MSE': mean_squared_error(y_test, y_pred_minmax),
    'R2': r2_score(y_test, y_pred_minmax)
}

for key, metrics in results.items():
    print(f"{key.upper()} -> MSE: {metrics['MSE']:.4f}, R²: {metrics['R2']:.4f}")

SEM_NORMALIZACAO -> MSE: 0.3789, R²: 0.0033
ZSCORE -> MSE: 0.3789, R²: 0.0033
MINMAX -> MSE: 0.3789, R²: 0.0033


### item i

In [19]:
pipeline_sem_preproc = Pipeline([
    ('regressor', LinearRegression())
])
pipeline_sem_preproc.fit(X_train, y_train)
y_pred1 = pipeline_sem_preproc.predict(X_test)
mse1 = mean_squared_error(y_test, y_pred1)
r2_1 = r2_score(y_test, y_pred1)

def remove_outliers_iqr(X, y, features):
    X_clean = X.copy()
    y_clean = y.copy()
    for col in features:
        Q1 = X_clean[col].quantile(0.25)
        Q3 = X_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        mask = (X_clean[col] >= (Q1 - 1.5 * IQR)) & (X_clean[col] <= (Q3 + 1.5 * IQR))
        X_clean = X_clean[mask]
        y_clean = y_clean[mask]
    return X_clean, y_clean

X_clean, y_clean = remove_outliers_iqr(X, y, numeric_features)

X_train, X_test, y_train, y_test = train_test_split(X_clean, y_clean, test_size=0.2, random_state=42)

pipeline_preproc = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])
pipeline_preproc.fit(X_train, y_train)
y_pred = pipeline_preproc.predict(X_test)

mse2 = mean_squared_error(y_test, y_pred)
r2_2 = r2_score(y_test, y_pred)

print("SEM PRÉ-PROCESSAMENTO")
print(f"MSE: {mse1:.4f}, R²: {r2_1:.4f}")

print("\nCOM PRÉ-PROCESSAMENTO (Remoção de outliers + Z-Score)")
print(f"MSE: {mse2:.4f}, R²: {r2_2:.4f}")

SEM PRÉ-PROCESSAMENTO
MSE: 0.3789, R²: 0.0033

COM PRÉ-PROCESSAMENTO (Remoção de outliers + Z-Score)
MSE: 0.3604, R²: 0.0028


### j. Embedding: BOW, TF-IDF, Word2Vec;

In [20]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

text_feature = 'cleaned_text'
numeric_features = ['word_count', 'char_count', 'unique_word_ratio', 
                    'link_count', 'avg_word_length']
X = result_df[[text_feature] + numeric_features]
y = np.log1p(result_df['shared_count'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

bow_transformer = ColumnTransformer(transformers=[
    ('text', CountVectorizer(), text_feature),
    ('num', StandardScaler(), numeric_features)
])

bow_pipeline = Pipeline([
    ('preproc', bow_transformer),
    ('regressor', LinearRegression())
])

bow_pipeline.fit(X_train, y_train)
y_pred_bow = bow_pipeline.predict(X_test)

tfidf_transformer = ColumnTransformer(transformers=[
    ('text', TfidfVectorizer(), text_feature),
    ('num', StandardScaler(), numeric_features)
])

tfidf_pipeline = Pipeline([
    ('preproc', tfidf_transformer),
    ('regressor', LinearRegression())
])

tfidf_pipeline.fit(X_train, y_train)
y_pred_tfidf = tfidf_pipeline.predict(X_test)

print("Bag of Words:")
print(f"MSE: {mean_squared_error(y_test, y_pred_bow):.4f}, R²: {r2_score(y_test, y_pred_bow):.4f}")

print("\nTF-IDF:")
print(f"MSE: {mean_squared_error(y_test, y_pred_tfidf):.4f}, R²: {r2_score(y_test, y_pred_tfidf):.4f}")


Bag of Words:
MSE: 4.0265, R²: -9.5910

TF-IDF:
MSE: 1.5508, R²: -3.0792


In [21]:
!pip install gensim

In [23]:
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer

from gensim.models import KeyedVectors
import gensim.downloader as api


w2v_model = api.load('word2vec-google-news-300')

class Word2VecVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, w2v_model):
        self.w2v_model = w2v_model
        self.dim = w2v_model.vector_size
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return np.array([self._average_word_vectors(doc) for doc in X])
    
    def _average_word_vectors(self, doc):
        words = str(doc).split()
        vectors = [self.w2v_model[word] for word in words if word in self.w2v_model]
        if not vectors:
            return np.zeros(self.dim)
        return np.mean(vectors, axis=0)



w2v_vectorizer = Word2VecVectorizer(w2v_model=w2v_model)

preprocessor = ColumnTransformer(transformers=[
    ('w2v', w2v_vectorizer, text_feature),
    ('num', StandardScaler(), numeric_features)
])

pipeline_w2v = Pipeline([
    ('preproc', preprocessor),
    ('regressor', LinearRegression())
])

pipeline_w2v.fit(X_train, y_train)
y_pred_w2v = pipeline_w2v.predict(X_test)

print("Word2Vec embedding:")
print(f"MSE: {mean_squared_error(y_test, y_pred_w2v):.4f}, R²: {r2_score(y_test, y_pred_w2v):.4f}")


Word2Vec embedding:
MSE: 0.3588, R²: 0.0564


### k. N-Gramas: unigramas, bigramas, trigramas;

In [24]:
def build_ngram_pipeline(ngram_range):
    vectorizer = CountVectorizer(ngram_range=ngram_range)

    preprocessor = ColumnTransformer(transformers=[
        ('vec', vectorizer, text_feature),
        ('num', StandardScaler(), numeric_features)
    ])

    pipeline = Pipeline([
        ('preproc', preprocessor),
        ('regressor', LinearRegression())
    ])
    
    return pipeline

In [25]:
for name, ngram in [('Unigrama', (1, 1)), ('Bigrama', (2, 2)), ('Trigrama', (3, 3))]:
    print(f'\nModelo: {name}')
    pipeline = build_ngram_pipeline(ngram)
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f'MSE: {mse:.4f}')
    print(f'R²: {r2:.4f}')



Modelo: Unigrama
MSE: 4.0265
R²: -9.5910

Modelo: Bigrama
MSE: 1.1192
R²: -1.9438

Modelo: Trigrama
MSE: 0.5491
R²: -0.4443


### l. Treinamento, Validação e Teste: Outer K-Fold Cross-Validation;

In [ ]:
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

no_scaler = 'passthrough'
zscore = StandardScaler()

bow = CountVectorizer()
tfidf = TfidfVectorizer()



def build_preprocessor(embedder, scaler):
    return ColumnTransformer([
        ('text', embedder, text_feature),
        ('num', scaler, numeric_features)
    ])


param_grid = [
    {
        'preproc__text': [bow, tfidf],
        'preproc__num': [no_scaler, zscore],
        'regressor': [Ridge()],
        'regressor__alpha': [0.1, 1.0, 10.0]
    }
]

In [57]:
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

outer_scores = []

for i, (train_idx, test_idx) in enumerate(outer_cv.split(X)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    pipeline = Pipeline([
        ('preproc', build_preprocessor(bow, no_scaler)),
        ('regressor', LinearRegression())
    ])

    grid = GridSearchCV(pipeline, param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
    grid.fit(X_train, y_train)

    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f"Fold {i+1}: MSE={mse:.4f} | R²={r2:.4f} | Best model: {grid.best_params_}")
    outer_scores.append((mse, r2, grid.best_params_))

mse_scores = [score[0] for score in outer_scores]
r2_scores = [score[1] for score in outer_scores]

print("\n==== Resultados finais ====")
print(f"MSE médio: {np.mean(mse_scores):.4f}")
print(f"R² médio: {np.mean(r2_scores):.4f}")

Fold 1: MSE=0.3212 | R²=0.1550 | Best model: {'preproc__num': 'passthrough', 'preproc__text': TfidfVectorizer(), 'regressor': Ridge(), 'regressor__alpha': 10.0}
Fold 2: MSE=0.3153 | R²=0.1617 | Best model: {'preproc__num': 'passthrough', 'preproc__text': TfidfVectorizer(), 'regressor': Ridge(), 'regressor__alpha': 10.0}
Fold 3: MSE=0.3247 | R²=0.1686 | Best model: {'preproc__num': 'passthrough', 'preproc__text': TfidfVectorizer(), 'regressor': Ridge(), 'regressor__alpha': 10.0}
Fold 4: MSE=0.3267 | R²=0.1570 | Best model: {'preproc__num': 'passthrough', 'preproc__text': TfidfVectorizer(), 'regressor': Ridge(), 'regressor__alpha': 10.0}
Fold 5: MSE=0.3280 | R²=0.1587 | Best model: {'preproc__num': 'passthrough', 'preproc__text': TfidfVectorizer(), 'regressor': Ridge(), 'regressor__alpha': 10.0}

==== Resultados finais ====
MSE médio: 0.3232
R² médio: 0.1602
